# Modelo de Redes Neuronales

El modelo de redes neuronales es construido a partir de un conjunto de datos con 5 clases:

    * wofs
    * bosque
    * wofs_bosque
    * cloud
    * ninguno

In [1]:
#!/usr/bin/env python
# coding: utf-8
#get_ipython().magic(u'matplotlib inline')
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras import regularizers
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import classification_report

import keras
#import matplotlib.pyplot as plt
from rasterio.plot import show
import xarray as xr

import rasterio
import glob
import os
import numpy as np
import pandas as pd

Using TensorFlow backend.


## Lectura del dataset

In [2]:
data = pd.read_csv('./satellite_dataset-con-nubes.csv')
data.head()

,blue,green,red,nir,swir1,swir2,wofs,bosque,wofs_bosque,cloud,ninguno
0,286.0,197.0,96.0,83.0,116.0,95.0,1,0,0,0,0
1,280.0,217.0,108.0,83.0,118.0,96.0,1,0,0,0,0
2,3011.0,2996.0,2900.0,4186.0,1683.0,1373.0,0,0,0,1,0
3,3576.0,3303.0,3313.0,3369.0,2637.0,2339.0,0,0,0,1,0
4,1398.0,1345.0,1311.0,1389.0,1633.0,1355.0,0,0,0,0,1


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 11 columns):
blue           1000000 non-null float64
green          1000000 non-null float64
red            1000000 non-null float64
nir            1000000 non-null float64
swir1          1000000 non-null float64
swir2          1000000 non-null float64
wofs           1000000 non-null int64
bosque         1000000 non-null int64
wofs_bosque    1000000 non-null int64
cloud          1000000 non-null int64
ninguno        1000000 non-null int64
dtypes: float64(6), int64(5)
memory usage: 83.9 MB


Se preparan los datos separando las characteristicas de los pixeles y las etiquetas de los pixeles. Luego se separan los datos en datos de entrenamiento y datos de prueba.

In [4]:
dataset_X = data.drop(columns=['wofs','bosque','wofs_bosque','cloud','ninguno'], axis=1)
dataset_X.head()
dataset_Y = data[['wofs','bosque','wofs_bosque','cloud','ninguno']]
dataset_Y.head()

X_train, X_test, y_train, y_test = train_test_split(dataset_X, dataset_Y, test_size=0.2)
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()


## DEFINICIÓN DEL MODELO



Se experimento con varios modelos distintos. Inicialmente se probó con modelos de una sola capa con una neurona, 4 neuronas, 6 neuronas, y 9 neuronas, agregando más neuronas después de este punto no se obtuvieron mejoras significativas. Luego se experimentó agregando más capas al modelo, la exactitud mejoró significativamente hasta la tercera capa. 

Se utilizó la función de activacion ReLU en todas las capas. La función softmax en la capa de salida asegura una clasificación mutuamente excluyente.

In [5]:
model = keras.Sequential([
#     keras.layers.Flatten(input_shape=(6,)),
    keras.layers.Dense(9,input_dim=6, activation='relu'),
    keras.layers.Dense(9, activation='relu'),
    keras.layers.Dense(9, activation='relu'),
    keras.layers.Dense(5, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
# Resumen de parámetros del modelo
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 9)                 63        
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 90        
_________________________________________________________________
dense_3 (Dense)              (None, 9)                 90        
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 50        
Total params: 293
Trainable params: 293
Non-trainable params: 0
_________________________________________________________________


In [6]:
print(X_train.shape)
# ENTRENAMIENTO
model.fit(X_train, y_train, epochs=2)

(800000, 6)
Epoch 1/2
800000/800000 [==============================] - 26s 33us/step - loss: 0.3816 - accuracy: 0.9330
Epoch 2/2
800000/800000 [==============================] - 27s 33us/step - loss: 0.1212 - accuracy: 0.9537


In [7]:
print("Saving model...")
model.save("model.h5")

Saving model...


## Resultados

* Clase 0 (wofs): Agua
* Clase 1 (bosque): Bosque
* Clase 2 (wofs_bosque): Agua o Bosque
* Clase 3 (cloud): Nubes
* Clase 4 (ninguno): Ninguna de las coverturas anteriores. Puedes ser repreentar una urbanización.

In [8]:
# EVALUACIÓN
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)


Test accuracy: 0.958869993686676


In [9]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred,axis=1)
y_pred_classes

array([3, 0, 3, ..., 4, 1, 0])

In [10]:
y_test_classes = np.argmax(y_test,axis=1)
y_test_classes

array([3, 0, 3, ..., 3, 1, 0])

In [11]:
confusion_matrix(y_test_classes, y_pred_classes)

array([[ 57854,     32,      0,    410,    598],
       [    27,  20078,      0,     36,    916],
       [   100,      7,      0,      0,     17],
       [   174,     16,      0, 105510,   1701],
       [   590,    594,      0,   3008,   8332]])

In [12]:
print(classification_report(y_test_classes, y_pred_classes))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98     58894
           1       0.97      0.95      0.96     21057
           2       0.00      0.00      0.00       124
           3       0.97      0.98      0.98    107401
           4       0.72      0.67      0.69     12524

    accuracy                           0.96    200000
   macro avg       0.73      0.72      0.72    200000
weighted avg       0.96      0.96      0.96    200000



/home/estudiante/fcrv/jupyter_notebooks/data_analisis_env_py3/lib/python3.5/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Análisis y Conclusiones

El modelo entrenado logra una exactitud del 96% sobre los datos de prueba. Comparandolo con el modelo sin la clase "cloud", este modelo logra una menor exactitud causado por una menor representación en los datos para las otras clases. La clase que fué más afectada fue "ninguno".

Las clases de "wofs", "bosque", y "cloud" son las mejores representadas en el dataset, y consequentement, logra mejores resultados en las predicciones del modelo, con una precisión del 98%.

La clase "ninguno" logra una precisión del 69%, y un recall del 75%, valores significativamente menores que aquellos obtenidos en el modelo sin nubes. A pesar de su poca representación en los datos, esta clase obtuvo metricas relativamente altas, estó parece indicar que esta clase posee caracteristicas faciles de identificar(Con valores del espectro de luz significativamente diferentes a los del bosque o el agua). Sin embargo, considerando que esta clase representa muchos terrenos, es posible que el modelo no generalice bien esta clase con otras imagenes satelitales.

Al igual que el modelo sin nubes, la clase wofs_bosque, siendo la menos representada, no logra ser predecida por el modelo. Esto se evidencia en una precisión y un recall igual a 0.

Para solucionar la poca representación de wofs_bosque, se podría buscar una imagen satelital con mayor superposición entre bosque y agua, como, por ejemplo, manglares, pantanos, o lagunas.